In [1]:
import torch
from torch import nn
from circuits import Circuit,Kinds
from learn import Trainer
from models import Solver
from models import State
from data import Preprocess


In [2]:
# Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

# torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(precision=3, linewidth=200)

Using cpu device


In [3]:
vr = Circuit()
vr.ring(Kinds.IVS,Kinds.R,100)
vr.elements[0].attr = 1
vr.elements[-1].i = 0.5

In [4]:
num_epochs = 1000
prev_loss = 0.1

input = Preprocess(vr)
model = Solver(input=input, state=State.Solve)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.1)
trainer = Trainer(model,optimizer,nn.MSELoss())
loss, _ = trainer.train()
print(f'init params: {model.attr}')
print(f'init loss: {loss.item()}')

epoch = 0

while(epoch < num_epochs):
    loss, _ = trainer.train()
    if(loss < 1e-10):
            break
    epoch += 1
    loss_change = abs(loss - prev_loss) / prev_loss
    prev_loss = loss

print(f'Done! at {epoch} passes, {loss.item()} loss')
print(f'solution = {model()}')
print(f"attributes = {model.attr}")

c:\Users\terry\OneDrive\Documents\GitHub\side_circuit\ml\circuits.py:104: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  M_scipy = nx.incidence_matrix(G=self.nx_graph(),oriented=True)


init params: Parameter containing:
tensor([1.000e+00, 1.788e-01, 5.379e-02, 7.287e-01, 8.598e-01, 1.212e-01, 5.185e-01, 8.139e-01, 7.806e-01, 5.233e-01, 6.883e-01, 1.068e-01, 3.116e-01, 6.913e-01, 1.222e-01, 4.805e-01, 6.322e-01,
        5.913e-01, 8.208e-01, 5.247e-02, 4.013e-03, 2.095e-01, 6.379e-01, 5.402e-01, 7.603e-01, 5.316e-02, 9.809e-02, 1.753e-01, 3.666e-01, 8.629e-01, 8.001e-01, 1.000e-12, 5.640e-01, 5.875e-01,
        8.977e-01, 4.570e-01, 8.654e-01, 5.983e-01, 7.678e-01, 2.935e-01, 8.184e-01, 6.134e-01, 3.874e-01, 1.574e-01, 4.928e-01, 3.549e-01, 6.429e-01, 2.315e-02, 2.286e-01, 6.871e-01, 3.193e-01,
        6.059e-01, 4.622e-01, 5.657e-01, 6.462e-01, 2.874e-01, 6.867e-01, 6.752e-01, 4.900e-01, 5.635e-01, 3.948e-01, 3.275e-01, 7.243e-01, 5.224e-01, 4.865e-01, 8.923e-01, 7.513e-01, 1.151e-02,
        7.198e-01, 4.796e-01, 5.251e-01, 6.675e-01, 1.000e-12, 1.000e-12, 2.974e-01, 5.610e-01, 2.473e-01, 7.192e-01, 4.852e-01, 7.854e-01, 8.581e-01, 2.633e-01, 4.647e-01, 3.162e-01, 7